### degrade_image_shift

In [1]:
import cv2
import numpy as np
import os
import json

In [2]:
# Папка с оригинальными изображениями
input_folder = "./content/DIV2K"

### filtered and  shifted

In [ ]:
output_folder = "./content/filtered_shifted" 
meta_info_path = f"{output_folder}/.meta_info.json"  # Файл для метаинформации

os.makedirs(output_folder, exist_ok=True)  # Создаем основную папку

# Функция ухудшения через смещение + снижение разрешения 
def degrade_image_transform(img, scale, shift_range):
    h, w = img.shape[:2]

    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)

    # 2 Генерация случайного сдвига
    shift_x = np.random.uniform(-shift_range, shift_range)
    shift_y = np.random.uniform(-shift_range, shift_range)

    # 3 Матрица сдвига
    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])

    # 4 Применяем сдвиг
    shifted = cv2.warpAffine(blurred, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # 5 Уменьшаем разрешение
    small = cv2.resize(shifted, (w // scale, h // scale), interpolation=cv2.INTER_LINEAR)

    return small, shift_x, shift_y

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png"))]

# Количество ухудшенных версий
num_versions = 10
meta_info = {}
scales = [2, 4, 8]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    meta_info[img_name] = {"original_size": img.shape[:2], "versions": []}

    for scl in scales:
        scale_folder = os.path.join(output_folder, f"K{scl}")
        os.makedirs(scale_folder, exist_ok=True)  #папка под масштаб


        for i in range(num_versions):
            degraded_img, shift_x, shift_y = degrade_image_transform(img, scl, scl-1)

            # Сохраняем ухудшенное изображение
            save_filename = f"{img_name[:-4]}_K{scl}_v{i}.png"
            save_path = os.path.join(scale_folder, save_filename)
            cv2.imwrite(save_path, degraded_img)

            # Сохраняем текстовый файл с метаинформацией рядом
            txt_path = os.path.join(scale_folder, f"{save_filename[:-4]}.txt")
            with open(txt_path, "w") as txt_file:
                txt_file.write(f"original_size: {img.shape[:2]}\n")
                txt_file.write(f"shift_x: {shift_x:.4f}\n")
                txt_file.write(f"shift_y: {shift_y:.4f}\n")
                txt_file.write(f"angle: 0\n")
                txt_file.write(f"scale: {scl}\n")

            # Сохраняем метаинформацию
            meta_info[img_name]["versions"].append({
                "file": f"K{scl}/{save_filename}",
                "shift_x": shift_x,
                "shift_y": shift_y,
                "angle": 0,
                "scale": scl
            })

# Сохраняем метаинформацию в JSON
with open(meta_info_path, "w") as f:
    json.dump(meta_info, f, indent=4)

print("Good")


Good


### filtered, shifted and rotated

In [4]:
output_folder = "./content/filtered_shifted_rotated"
meta_info_path = f"{output_folder}/.meta_info.json"  # Файл для метаинформации

os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

# Функция ухудшения 
def degrade_image_transform(img, scale, shift_range, rotation_range=5):
    h, w = img.shape[:2]
    
    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)
    
    # 2 Генерируем случайные параметры
    shift_x = np.random.uniform(-shift_range, shift_range)  # Сдвиг по X
    shift_y = np.random.uniform(-shift_range, shift_range)  # Сдвиг по Y
    angle = np.random.uniform(-rotation_range, rotation_range)  # Поворот в градусах
    
    # 3 Генерируем координаты для выходного изображения 
    x_out, y_out = np.meshgrid(np.arange(w), np.arange(h))
    center = (w // 2, h // 2)
    
    # 4 Применяем преобразование подобия
    angle_rad = np.deg2rad(angle)  # Переводим градусы в радианы
    cos_fi, sin_fi = np.cos(angle_rad), np.sin(angle_rad)
    
    x_in = (cos_fi * (x_out - center[0]) + sin_fi * (y_out - center[1])) + center[0] + shift_x
    y_in = (-sin_fi * (x_out - center[0]) + cos_fi * (y_out - center[1])) + center[1] + shift_y
    
    # 5 Используем `cv2.remap()` для трансформации
    map_x, map_y = x_in.astype(np.float32), y_in.astype(np.float32)
    transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    
    # 6 Уменьшаем изображение
    degraded = cv2.resize(transformed, (w // scale, h // scale), interpolation=cv2.INTER_LINEAR)
    
    return degraded, shift_x, shift_y, angle

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png"))]

# Количество ухудшенных версий для каждого изображения
num_versions = 10
meta_info = {}
scales = [2, 4, 8]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    meta_info[img_name] = {"original_size": img.shape[:2], "versions": []}

    for scl in scales:
        scale_folder = os.path.join(output_folder, f"K{scl}")
        os.makedirs(scale_folder, exist_ok=True)  #папка под масштаб


        for i in range(num_versions):
            degraded_img, shift_x, shift_y, angle = degrade_image_transform(img, scl, scl-1)

            # Сохраняем ухудшенное изображение
            save_filename = f"{img_name[:-4]}_K{scl}_v{i}.png"
            save_path = os.path.join(scale_folder, save_filename)
            cv2.imwrite(save_path, degraded_img)

            # Сохраняем текстовый файл с метаинформацией рядом
            txt_path = os.path.join(scale_folder, f"{save_filename[:-4]}.txt")
            with open(txt_path, "w") as txt_file:
                txt_file.write(f"original_size: {img.shape[:2]}\n")
                txt_file.write(f"shift_x: {shift_x:.4f}\n")
                txt_file.write(f"shift_y: {shift_y:.4f}\n")
                txt_file.write(f"angle: {angle}\n")
                txt_file.write(f"scale: {scl}\n")
                
            # Сохраняем метаинформацию
            meta_info[img_name]["versions"].append({
                "file": f"K{scl}/{save_filename}",
                "shift_x": shift_x,
                "shift_y": shift_y,
                "angle": angle,
                "scale": scl
            })

with open(meta_info_path, "w") as f:
    json.dump(meta_info, f, indent=4)

print("Good")

Good


### filtered, shifted, rotated and scaled

In [6]:
output_folder = "./content/filtered_shifted_rotated_scaled"
meta_info_path = f"{output_folder}/.meta_info.json"  # Файл для метаинформации

os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

# Функция ухудшения 
def degrade_image_transform(img, scale, shift_range, rotation_range=5):
    h, w = img.shape[:2]
    
    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)
    
    # 2 Генерация случайных параметров
    shift_x = np.random.uniform(-shift_range, shift_range)  # Сдвиг по X
    shift_y = np.random.uniform(-shift_range, shift_range)  # Сдвиг по Y
    angle = np.random.uniform(-rotation_range, rotation_range)  # Поворот в градусах
    # k = np.random.uniform(0.9, 1.1)  # Коэффициент масштаба
    alpha = np.random.uniform(0.9, 1.1)  # Коэффициент масштаба
    
    # 3 Применяем сдвиг и поворот
    angle_rad = np.deg2rad(angle)  # Переводим градусы в радианы
    cos_fi, sin_fi = np.cos(angle_rad), np.sin(angle_rad)
    
    center = (w // 2, h // 2)  # Центр изображения
    x_out, y_out = np.meshgrid(np.arange(w), np.arange(h))  # Координаты в новом пространстве
    
    x_in = (cos_fi * (x_out - center[0]) + sin_fi * (y_out - center[1])) + center[0] + shift_x
    y_in = (-sin_fi * (x_out - center[0]) + cos_fi * (y_out - center[1])) + center[1] + shift_y
    
    # 4 Используем `cv2.remap()` для трансформации
    map_x, map_y = x_in.astype(np.float32), y_in.astype(np.float32)
    transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    
    # 5 Уменьшаем изображение в (alpha / scale) раз
    small = cv2.resize(transformed, (int(w * alpha / scale), int(h * alpha / scale)), interpolation=cv2.INTER_LINEAR)
    
    return small, shift_x, shift_y, angle, alpha / scale

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png"))]

# Количество ухудшенных версий для каждого изображения
num_versions = 10
meta_info = {}
scales = [2, 4, 8]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    meta_info[img_name] = {"original_size": img.shape[:2], "versions": []}

    for scl in scales:
        scale_folder = os.path.join(output_folder, f"K{scl}")
        os.makedirs(scale_folder, exist_ok=True)  #папка под масштаб

        for i in range(num_versions):
            degraded_img, shift_x, shift_y, angle, scale = degrade_image_transform(img, scl, scl-1)

            # Сохраняем ухудшенное изображение
            save_filename = f"{img_name[:-4]}_K{scl}_v{i}.png"
            save_path = os.path.join(scale_folder, save_filename)
            cv2.imwrite(save_path, degraded_img)

            # Сохраняем текстовый файл с метаинформацией рядом
            txt_path = os.path.join(scale_folder, f"{save_filename[:-4]}.txt")
            with open(txt_path, "w") as txt_file:
                txt_file.write(f"original_size: {img.shape[:2]}\n")
                txt_file.write(f"shift_x: {shift_x:.4f}\n")
                txt_file.write(f"shift_y: {shift_y:.4f}\n")
                txt_file.write(f"angle: {angle}\n")
                txt_file.write(f"scale: {scale}\n")

            # Сохраняем метаинформацию
            meta_info[img_name]["versions"].append({
                "file": f"K{scl}/{save_filename}",                
                "shift_x": shift_x,
                "shift_y": shift_y,
                "angle": angle,
                "scale": scale
            })

with open(meta_info_path, "w") as f:
    json.dump(meta_info, f, indent=4)

print("Good")


Good


### filtered, shifted and scaled

In [ ]:
output_folder = "./content/filtered_shifted_scaled"
meta_info_path = f"{output_folder}/.meta_info.json"  # Файл для метаинформации

os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

def degrade_image_transform(img, scale, shift_range=5):
    h, w = img.shape[:2]

    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)

    # 2 Генерация случайного сдвига
    shift_x = np.random.uniform(-shift_range, shift_range)
    shift_y = np.random.uniform(-shift_range, shift_range) 
    alpha = np.random.uniform(0.9, 1.1)  # Коэффициент масштаба

    # 3 Матрица сдвига
    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])

    # 4 Применяем сдвиг
    shifted = cv2.warpAffine(blurred, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # 5 Уменьшаем разрешение в (alpha / scale) раз
    small = cv2.resize(shifted, (int(w * alpha / scale), int(h * alpha / scale)), interpolation=cv2.INTER_LINEAR)

    return small, shift_x, shift_y, alpha / scale


# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png"))]

# Количество ухудшенных версий для каждого изображения
num_versions = 10
meta_info = {}
scales = [2, 4, 8]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    meta_info[img_name] = {"original_size": img.shape[:2], "versions": []}

    for scl in scales:
        scale_folder = os.path.join(output_folder, f"K{scl}")
        os.makedirs(scale_folder, exist_ok=True)  #папка под масштаб

        for i in range(num_versions):
            degraded_img, shift_x, shift_y, scale = degrade_image_transform(img, scl, scl-1)

            # Сохраняем ухудшенное изображение
            save_filename = f"{img_name[:-4]}_K{scl}_v{i}.png"
            save_path = os.path.join(scale_folder, save_filename)
            cv2.imwrite(save_path, degraded_img)

            # Сохраняем текстовый файл с метаинформацией рядом
            txt_path = os.path.join(scale_folder, f"{save_filename[:-4]}.txt")
            with open(txt_path, "w") as txt_file:
                txt_file.write(f"original_size: {img.shape[:2]}\n")
                txt_file.write(f"shift_x: {shift_x:.4f}\n")
                txt_file.write(f"shift_y: {shift_y:.4f}\n")
                txt_file.write(f"angle: {0}\n")
                txt_file.write(f"scale: {scale}\n")

            # Сохраняем метаинформацию
            meta_info[img_name]["versions"].append({
                "file": f"K{scl}/{save_filename}",                
                "shift_x": shift_x,
                "shift_y": shift_y,
                "angle": 0,
                "scale": scale
            })

with open(meta_info_path, "w") as f:
    json.dump(meta_info, f, indent=4)

print("Good")


Good
